In [1]:
# Connect colab with google drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
# Importing required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
# Importing Dataset

housing_data = pd.read_csv("/content/drive/MyDrive/Datasets/USA_Housing.csv")
print(housing_data.head())
print("Column names : ",housing_data.columns)
print("Dataset info : ",housing_data.info())

   Avg. Area Income  Avg. Area House Age  Avg. Area Number of Rooms  \
0      79545.458574             5.682861                   7.009188   
1      79248.642455             6.002900                   6.730821   
2      61287.067179             5.865890                   8.512727   
3      63345.240046             7.188236                   5.586729   
4      59982.197226             5.040555                   7.839388   

   Avg. Area Number of Bedrooms  Area Population         Price  \
0                          4.09     23086.800503  1.059034e+06   
1                          3.09     40173.072174  1.505891e+06   
2                          5.13     36882.159400  1.058988e+06   
3                          3.26     34310.242831  1.260617e+06   
4                          4.23     26354.109472  6.309435e+05   

                                             Address  
0  208 Michael Ferry Apt. 674\nLaurabury, NE 3701...  
1  188 Johnson Views Suite 079\nLake Kathleen, CA...  
2  9127 Eli

In [4]:
# Data preprocessing

# Since we cannot process address data we drop that from data set

data = housing_data.drop(columns=["Address"],axis=1)
print(data.head())

print(data.columns)

   Avg. Area Income  Avg. Area House Age  Avg. Area Number of Rooms  \
0      79545.458574             5.682861                   7.009188   
1      79248.642455             6.002900                   6.730821   
2      61287.067179             5.865890                   8.512727   
3      63345.240046             7.188236                   5.586729   
4      59982.197226             5.040555                   7.839388   

   Avg. Area Number of Bedrooms  Area Population         Price  
0                          4.09     23086.800503  1.059034e+06  
1                          3.09     40173.072174  1.505891e+06  
2                          5.13     36882.159400  1.058988e+06  
3                          3.26     34310.242831  1.260617e+06  
4                          4.23     26354.109472  6.309435e+05  
Index(['Avg. Area Income', 'Avg. Area House Age', 'Avg. Area Number of Rooms',
       'Avg. Area Number of Bedrooms', 'Area Population', 'Price'],
      dtype='object')


In [6]:
# Setting independent variable
x = data.drop(columns = ["Price"],axis = 1)
print(x.head())

# Setting dependent variable
y = data["Price"]
print(y)

# Spliting dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

   Avg. Area Income  Avg. Area House Age  Avg. Area Number of Rooms  \
0      79545.458574             5.682861                   7.009188   
1      79248.642455             6.002900                   6.730821   
2      61287.067179             5.865890                   8.512727   
3      63345.240046             7.188236                   5.586729   
4      59982.197226             5.040555                   7.839388   

   Avg. Area Number of Bedrooms  Area Population  
0                          4.09     23086.800503  
1                          3.09     40173.072174  
2                          5.13     36882.159400  
3                          3.26     34310.242831  
4                          4.23     26354.109472  
0       1.059034e+06
1       1.505891e+06
2       1.058988e+06
3       1.260617e+06
4       6.309435e+05
            ...     
4995    1.060194e+06
4996    1.482618e+06
4997    1.030730e+06
4998    1.198657e+06
4999    1.298950e+06
Name: Price, Length: 5000, dtype: fl

In [8]:
# Standardizing features for NN
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
print("S x_train : ",x_train)
print("S y_train : ",y_train)

S x_train :  [[-1.24020578e+00 -2.02767161e+00  1.95462007e+00  3.23711446e-01
   7.57543586e-01]
 [-5.84004848e-01  5.24312814e-01  6.54791707e-01 -6.47558147e-01
   1.11053001e+00]
 [ 8.13832570e-01 -5.70507901e-01  1.24429714e+00 -7.12853582e-01
  -9.37736647e-01]
 ...
 [-2.22487078e-01  5.41698513e-01 -3.69741691e-01 -6.88367793e-01
   1.26855496e-01]
 [-9.19631119e-01  1.43796367e+00  2.25970126e+00  2.74739870e-01
   1.40171401e+00]
 [-6.90243352e-01 -7.64877051e-02  8.85481254e-01  1.67859172e+00
   5.75204193e-04]]
S y_train :  1840    8.814461e+05
2115    1.375771e+06
4437    1.137069e+06
1146    9.289500e+05
2486    1.392084e+06
            ...     
4426    1.023944e+06
466     1.223101e+06
3092    1.318598e+06
3772    1.708631e+06
860     1.060898e+06
Name: Price, Length: 3500, dtype: float64


In [9]:
# Build the deep learning model
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(x_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)  # Output layer for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
history = model.fit(x_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)
print("history : ",history)

# Evaluate the model
y_pred = model.predict(x_test)

# Calculate R-squared (R2) score
r2 = r2_score(y_test, y_pred)
print("R-squared (R2) Score:", r2)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


Epoch 1/100
88/88 [==============================] - 2s 8ms/step - loss: 1635276423168.0000 - mae: 1228030.7500 - val_loss: 1638324240384.0000 - val_mae: 1228760.3750
Epoch 2/100
88/88 [==============================] - 0s 4ms/step - loss: 1633247952896.0000 - mae: 1227242.3750 - val_loss: 1632766132224.0000 - val_mae: 1226628.5000
Epoch 3/100
88/88 [==============================] - 0s 4ms/step - loss: 1618005852160.0000 - mae: 1221425.8750 - val_loss: 1603305472000.0000 - val_mae: 1215521.7500
Epoch 4/100
88/88 [==============================] - 0s 4ms/step - loss: 1562460553216.0000 - mae: 1200361.8750 - val_loss: 1516001558528.0000 - val_mae: 1182490.7500
Epoch 5/100
88/88 [==============================] - 1s 6ms/step - loss: 1430149529600.0000 - mae: 1148957.6250 - val_loss: 1333896675328.0000 - val_mae: 1111000.6250
Epoch 6/100
88/88 [==============================] - 0s 3ms/step - loss: 1192712863744.0000 - mae: 1050462.7500 - val_loss: 1043017957376.0000 - val_mae: 985592.7500